In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("📌 Using device:", device)

In [ ]:
# -------------------------------
# 1. 전처리
# -------------------------------
root_dir = os.path.expanduser("./archive/chest_xray")
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 흑백(1채널)로 변환
    transforms.Resize((128, 128)),  # 128 by 128 크기로 변환
    transforms.ToTensor()  # 이미지를 PyTorch Tensor로 변환
])
input_dim = 128 * 128  # 모델의 입력 차원

# train/val/test dataset 구성
trainval_ds = datasets.ImageFolder(os.path.join(root_dir, 'train'), transform=transform)  # train+val
test_ds = datasets.ImageFolder(os.path.join(root_dir, 'test'),  transform=transform)  # test

# train/val 분할
train_size = int(0.8 * len(trainval_ds))  # train set의 사이즈
val_size = len(trainval_ds) - train_size  # val set의 사이즈
train_ds, val_ds = random_split(trainval_ds, [train_size, val_size], generator=torch.Generator().manual_seed(42))
print("train:", train_size, "val:", val_size, "test:", len(test_ds))

# DataLoader 구성 (batch_size=32, iteration마다 32개 이미지 처리)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)  # shuffle: 과적합 방지 및 일반화 성능 향상 
val_loader = DataLoader(val_ds, batch_size=32)  
test_loader = DataLoader(test_ds, batch_size=32)

In [ ]:
# -------------------------------
# 3. Logistic 모델 정의
# -------------------------------
class LogisticModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)  # binary classficaiton

    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # sigmoid 함수로 확률 출력 (0 ~ 1)

In [ ]:
# -------------------------------
# 2. DeepNN 모델 정의
# -------------------------------
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# -------------------------------
# 3. 학습 진행하는 함수 (학습)
# -------------------------------
def train(model, loader, optimizer, criterion):
    model.train()  # 학습 모드 활성화
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in loader:
        images = images.view(images.size(0), -1).to(device)  # Flatten
        labels = labels.float().unsqueeze(1).to(device)  # [32] → [32,1]

        optimizer.zero_grad()              # 기울기 초기화
        outputs = model(images)            # forward
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()                    # 역전파
        optimizer.step()                   # 파라미터 업데이트

        running_loss += loss.item() * labels.size(0)
        preds = (outputs >= 0.5).float()   # 시그모이드 → 0 또는 1
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
# -------------------------------
# 3. 학습 진행하는 함수 (검증)
# -------------------------------
def validate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)

            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
# -------------------------------
# 4. 테스트 진행
# -------------------------------
def evaluate(model, loader):
    model.eval()  # 평가 모드
    correct, total = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1).to(device)  # Flatten
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)

            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy


In [ ]:
# -------------------------------
# 5. 학습 함수를 통해 학습 
# -------------------------------

# 저장용 리스트
model_names = ["Logistic", "DeepNN"]
train_accs, val_accs, test_accs = [], [], []
epochs = 2

# Logistic
model1 = LogisticModel(input_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

for epoch in range(epochs):
    train_loss, train_acc = train(model1, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model1, val_loader, criterion)
    print(f"[Logistic][Epoch {epoch+1}] Train Acc: {train_acc:.2f}, Val Acc: {val_acc:.2f}")

logistic_train_acc = train_acc
logistic_val_acc = val_acc

train_accs.append(logistic_train_acc)
val_accs.append(logistic_val_acc)

# DeepNN
model2 = DeepNN(input_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

for epoch in range(epochs):
    train_loss, train_acc = train(model2, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model2, val_loader, criterion)
    print(f"[DNN][Epoch {epoch+1}] Train Acc: {train_acc:.2f}, Val Acc: {val_acc:.2f}")

dnn_train_acc = train_acc
dnn_val_acc = val_acc

train_accs.append(dnn_train_acc)
val_accs.append(dnn_val_acc)

In [ ]:
# -------------------------------
# 6. 테스트를 통해 정확도 도출
# -------------------------------

# Logistic
logistic_test_acc = evaluate(model1, test_loader)
test_accs.append(logistic_test_acc)

# DeepNN
dnn_test_acc = evaluate(model2, test_loader)
test_accs.append(dnn_test_acc)

In [ ]:
# -------------------------------
# 7. 시각화
# -------------------------------
import numpy as np

x = np.arange(len(model_names))
bar_width = 0.2

plt.figure(figsize=(5, 5))
plt.bar(x - bar_width, train_accs, width=bar_width, label="Train")
plt.bar(x, val_accs, width=bar_width, label="Val")
plt.bar(x + bar_width, test_accs, width=bar_width, label="Test")

plt.xticks(x, model_names)
plt.ylabel("Accuracy")
plt.ylim(0.5, 1.0)
plt.title("Chest X-ray Binary Classfication")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()